In [1]:
### Download renewables in new buildings data for Landkreise
### from inkar.de and compile into csv with Regions as rows
### and years as columns.

In [2]:
#https://www.regionalstatistik.de/genesis/online?operation=previous&levelindex=3&step=2&titel=Tabellenaufbau&levelid=1655207886087&levelid=1655207857464#abreadcrumb
#Fertigstellungen neuer Wohngebäude und Nichtwohngebäude sowie Wohnungen in Wohngebäuden nach Zahl der Wohnungen und vorwiegender Art der Beheizung - Jahressumme
# downloaded as "flat"

In [3]:
# import necessary libraries
import numpy as np
import pandas as pd

In [4]:
## Dataset: regionalstatistik

#helper function to move values around 
#needed because of structural problems with the raw data
def move_to_third_section(df,title,value):
    second_section = "2_" + title;
    third_section = "3_" + title;

    if "Merkmal" in title:
        df[third_section] = np.where((df[second_section] == value), df[second_section],df[third_section])
        #df[second_section] = np.where((df[second_section] == value),"WOHNGEBAUDE GESAMT",df[third_section])

    elif "Auspraegung" in title:
        merkmal = second_section.replace("Auspraegung","Merkmal")
        df[third_section] = np.where((df[merkmal] == value), df[second_section],df[third_section])
        #df[second_section] = np.where((df[merkmal] == value),"WOHNGEBAUDE GESAMT",df[third_section])

    else:
        print("unknown title: ", title)

    return df

In [5]:
#preprocess dataset
def preprocess_dataset_kreise(df):

    #set AGS Germany to 0 (so we only have numvers)
    df["1_Auspraegung_Code"] = df["1_Auspraegung_Code"].replace({"DG":"0"})    
    df["1_Auspraegung_Code"] = pd.to_numeric(df["1_Auspraegung_Code"])

    #clean up multi level format of raw data 
    df["2_Auspraegung_Label"] = df["2_Auspraegung_Label"].replace({"Insgesamt":'Wohnungen Insgesamt'})  
    #df["2_Auspraegung_Label"] = df["2_Auspraegung_Label"].str.replace('Insgesamt','Wohnungen Insgesamt',regex=True)
    df.loc[df["2_Auspraegung_Code"] == "INSGESAMT","2_Auspraegung_Label"] = "Wohngebäude Insgesamt"

    #rename titles
    titles = ["Merkmal_Code","Merkmal_Label","Auspraegung_Code","Auspraegung_Label"]

    for title in titles:
        if "Code" in title:
            df = move_to_third_section(df,title,"BAUPHE")
        elif "Label" in title:
            df = move_to_third_section(df,title,"Primär verwendete Heizenergie")

    for title in reversed(titles):
        title = "2_" + title
        df[title] = np.where((df["2_Merkmal_Code"] == "BAUPHE"),"Wohngebäude Insgesamt",df[title])

    #rename columns
    df.columns = df.columns.str.replace("1_", "Ort_")
    df.columns = df.columns.str.replace("2_", "Wohnort_")
    df.columns = df.columns.str.replace("3_", "Heizenergie_")


    #make building counts numeric
    building_counts = ["BAU017__Wohngebaeude_insgesamt__Anzahl","WOHN04__Wohnungen_in_Wohngebaeuden_(ohne_Wohnheime)__Anzahl","BAU018__Nichtwohngebaeude_insgesamt__Anzahl"]
    
    for building in building_counts:
        df[building] = df[building].str.replace('-','0',regex=True)
        df[building].fillna(0,inplace=True)
        df[building] = pd.to_numeric(df[building])


    #set total number of buildings for each year and place
    #needed to calculate proportions
    years = list(range(2016, 2021))
    orte = df["Ort_Auspraegung_Code"].unique()

    for ort in orte:
        for year in years:
            isLocation = (df["Ort_Auspraegung_Code"] == ort)
            isYear = (df["Zeit"] == year)

            gesamt = df.loc[isLocation & isYear & (df["Heizenergie_Auspraegung_Code"] == "INSGESAMT")].copy()
            
            gesamt_wohngebaeude = gesamt.loc[(gesamt["Wohnort_Auspraegung_Label"] == "Wohngebäude Insgesamt"),"BAU017__Wohngebaeude_insgesamt__Anzahl"]
            gesamt_wohnungen = gesamt.loc[(gesamt["Wohnort_Auspraegung_Label"] == "Wohnungen Insgesamt"),"WOHN04__Wohnungen_in_Wohngebaeuden_(ohne_Wohnheime)__Anzahl"]

            if(gesamt_wohngebaeude.size == 1 and gesamt_wohnungen.size == 1):
                df.loc[isLocation & isYear,"Wohngebäude_Gesamt"] = int(gesamt_wohngebaeude.item())
                df.loc[isLocation & isYear,"Wohnungen_Gesamt"] = int(gesamt_wohnungen.item())
            else:
                print("error: gesamt is wrong",gesamt)

    #calculate proportions
    df["Wohngebäude_Anteil"] = df["BAU017__Wohngebaeude_insgesamt__Anzahl"] / df["Wohngebäude_Gesamt"] * 100
    df["Wohnungen_Anteil"] = df["WOHN04__Wohnungen_in_Wohngebaeuden_(ohne_Wohnheime)__Anzahl"] / df["Wohnungen_Gesamt"] * 100

    #Drop unneccessary columns to save space
    df.drop(['Statistik_Label','Ort_Merkmal_Label'], axis=1, inplace = True)

    return df

In [6]:
#create pivoted data table
#         viz : df dataframe slice of your data, for example a specific landkreis
#        asProportion : bool, optional
#            if true, the visualisation will use proportion. otherwise, the total number is used. default is true
#        useWohnungen : bool, optional
#            if true, flats will be counted. otherwise, buildings will be counted (which maybe have multiple flats in them). default is true
def create_pivot_tables(df_p,asProportion = True,useWohnungen = True):

    df = df_p.copy()
    
    #rename columns
    df.rename({"Zeit": "year","BAU017__Wohngebaeude_insgesamt__Anzahl":"buildings","Heizenergie_Auspraegung_Label":"energy",
        "Wohngebäude_Anteil":"buildings_anteil","Wohnungen_Anteil":"flats_anteil",
        "WOHN04__Wohnungen_in_Wohngebaeuden_(ohne_Wohnheime)__Anzahl":"flats"}, axis=1,inplace=True)

    # dont add gesamtenergie to plot
    df = df[df["energy"] != "Wohngebäude Insgesamt"]
    df = df[df["energy"] != "Wohnungen Insgesamt"]
    df = df[df["energy"] != "Insgesamt"]

    if useWohnungen:
        column="flats"
        
        #for wohnungen: do not differ between 1, 2 or 3 or more flats
        df= df[df["Wohnort_Auspraegung_Label"] == "Wohnungen Insgesamt"]
    
    else:
        column='buildings'
        df= df[df["Wohnort_Auspraegung_Label"] == "Wohngebäude Insgesamt"]
    
    if asProportion:
        column += "_anteil"

    #pivot data
    orte = df["Ort_Auspraegung_Code"].unique()

    pivot_total = pd.DataFrame()

    for ort in orte:
        snippet = df[df["Ort_Auspraegung_Code"] == ort]

        temp = pd.pivot_table(snippet, index='year',columns='energy', values=column)
        temp = temp.reset_index(level=0)

        temp["ort_ags"] = ort
        temp["ort_name"]= str(snippet["Ort_Auspraegung_Label"].iloc[0])

        pivot_total = pd.concat([pivot_total, temp])

    #sum up renewables and fossils
    renewables = ["Solarthermie","Geothermie","Umweltthermie (Luft/Wasser)","Holz","Biogas/Biomethan","Sonstige Biomasse","Keine Energie (einschl. Passivhaus)"]
    fossils = ["Gas","Strom","Öl","Fernwärme/Fernkälte","Sonstige Heizenergie"]

    pivot_total["renewables"] = pivot_total[renewables].sum(axis=1)
    pivot_total["fossils"] = pivot_total[fossils].sum(axis=1)


    
    pivot_total.to_csv("data/newbuildings_pivot_"+column+".csv")
    return pivot_total

In [7]:
#load dataset from local file
nb_kreise = pd.read_csv("data/newbuildings_kreise.csv",delimiter=";", encoding='latin1', low_memory=False)

#preprocessing
nb_kreise = preprocess_dataset_kreise(nb_kreise)
nb_kreise

,Statistik_Code,Zeit_Code,Zeit_Label,Zeit,Ort_Merkmal_Code,Ort_Auspraegung_Code,Ort_Auspraegung_Label,Wohnort_Merkmal_Code,Wohnort_Merkmal_Label,Wohnort_Auspraegung_Code,...,Heizenergie_Merkmal_Label,Heizenergie_Auspraegung_Code,Heizenergie_Auspraegung_Label,BAU017__Wohngebaeude_insgesamt__Anzahl,WOHN04__Wohnungen_in_Wohngebaeuden_(ohne_Wohnheime)__Anzahl,BAU018__Nichtwohngebaeude_insgesamt__Anzahl,Wohngebäude_Gesamt,Wohnungen_Gesamt,Wohngebäude_Anteil,Wohnungen_Anteil
0,31121,JAHR,Jahr,2020,KREISE,0,Deutschland,Wohngebäude Insgesamt,Wohngebäude Insgesamt,Wohngebäude Insgesamt,...,Primär verwendete Heizenergie,BAUPHE00,Keine Energie (einschl. Passivhaus),111,0,12960,112935.0,261124.0,0.098287,0.000000
1,31121,JAHR,Jahr,2020,KREISE,0,Deutschland,Wohngebäude Insgesamt,Wohngebäude Insgesamt,Wohngebäude Insgesamt,...,Primär verwendete Heizenergie,BAUPHE02,Öl,776,0,272,112935.0,261124.0,0.687121,0.000000
2,31121,JAHR,Jahr,2020,KREISE,0,Deutschland,Wohngebäude Insgesamt,Wohngebäude Insgesamt,Wohngebäude Insgesamt,...,Primär verwendete Heizenergie,BAUPHE03,Gas,44037,0,5014,112935.0,261124.0,38.993226,0.000000
3,31121,JAHR,Jahr,2020,KREISE,0,Deutschland,Wohngebäude Insgesamt,Wohngebäude Insgesamt,Wohngebäude Insgesamt,...,Primär verwendete Heizenergie,BAUPHE04,Strom,1606,0,780,112935.0,261124.0,1.422057,0.000000
4,31121,JAHR,Jahr,2020,KREISE,0,Deutschland,Wohngebäude Insgesamt,Wohngebäude Insgesamt,Wohngebäude Insgesamt,...,Primär verwendete Heizenergie,BAUPHE05,Fernwärme/Fernkälte,9053,0,1648,112935.0,261124.0,8.016115,0.000000
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
174845,31121,JAHR,Jahr,2016,KREISE,16077,"Altenburger Land, Kreis",GEBWH1,Gebäude nach Anzahl der Wohnungen,WHGZHL03UM,...,Primär verwendete Heizenergie,BAUPHE09,Holz,0,0,0,58.0,101.0,0.000000,0.000000
174846,31121,JAHR,Jahr,2016,KREISE,16077,"Altenburger Land, Kreis",GEBWH1,Gebäude nach Anzahl der Wohnungen,WHGZHL03UM,...,Primär verwendete Heizenergie,BAUPHE10,Biogas/Biomethan,0,0,0,58.0,101.0,0.000000,0.000000
174847,31121,JAHR,Jahr,2016,KREISE,16077,"Altenburger Land, Kreis",GEBWH1,Gebäude nach Anzahl der Wohnungen,WHGZHL03UM,...,Primär verwendete Heizenergie,BAUPHE11,Sonstige Biomasse,0,0,0,58.0,101.0,0.000000,0.000000
174848,31121,JAHR,Jahr,2016,KREISE,16077,"Altenburger Land, Kreis",GEBWH1,Gebäude nach Anzahl der Wohnungen,WHGZHL03UM,...,Primär verwendete Heizenergie,BAUPHE12,Sonstige Heizenergie,0,0,0,58.0,101.0,0.000000,0.000000


In [8]:
# create_pivot_tables_
#         viz : df dataframe slice of your data, for example a specific landkreis
#        asProportion : bool, optional
#            if true, the visualisation will use proportion. otherwise, the total number is used. default is true
#        useWohnungen : bool, optional
#            if true, flats will be counted. otherwise, buildings will be counted (which maybe have multiple flats in them). default is true
pivot = create_pivot_tables(nb_kreise,False,True)
pivot

energy,year,Biogas/Biomethan,Fernwärme/Fernkälte,Gas,Geothermie,Holz,Keine Energie (einschl. Passivhaus),Solarthermie,Sonstige Biomasse,Sonstige Heizenergie,Strom,Umweltthermie (Luft/Wasser),Öl,ort_ags,ort_name,renewables,fossils
0,2016,919,42350,110439,12661,11690,123,1094,1135,911,1310,37200,1619,0,Deutschland,64822,156629
1,2017,1025,43993,105864,12543,13210,133,1127,1152,821,1243,46018,1660,0,Deutschland,75208,153581
2,2018,693,50748,98898,13934,14300,192,1488,829,1686,1569,52501,1479,0,Deutschland,83937,154380
3,2019,963,55745,98586,14778,12562,174,1157,857,1352,1841,56646,1502,0,Deutschland,87137,159026
4,2020,1104,60312,98726,15368,12026,129,1077,1084,1041,2424,66481,1352,0,Deutschland,97269,163855
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
0,2016,0,3,77,6,0,0,1,1,0,0,13,0,16077,"Altenburger Land, Kreis",21,80
1,2017,0,3,31,6,4,0,2,1,0,0,21,0,16077,"Altenburger Land, Kreis",34,34
2,2018,0,5,26,9,2,0,0,0,0,6,24,0,16077,"Altenburger Land, Kreis",35,37
3,2019,0,110,24,3,1,0,1,0,0,1,17,1,16077,"Altenburger Land, Kreis",22,136


In [9]:
#change year column to string to facilitate renaming columns after second pivot
pivot['year'] = pivot['year'].astype(str)

#pivot dataframe to have both energy types and years as columns
piv_df = pivot.pivot(index=['ort_ags','ort_name'],columns='year',values=["renewables","fossils","Solarthermie","Geothermie","Umweltthermie (Luft/Wasser)","Holz","Biogas/Biomethan","Sonstige Biomasse","Keine Energie (einschl. Passivhaus)","Gas","Strom","Öl","Fernwärme/Fernkälte","Sonstige Heizenergie"])
piv_df

renewables                       \
year                                            2016   2017   2018   2019   
ort_ags  ort_name                                                           
0        Deutschland                           64822  75208  83937  87137   
1          Schleswig-Holstein                   2161   2644   2519   2876   
2          Hamburg                               562    550    874    656   
3          Niedersachsen                        4011   4348   5150   5363   
4          Bremen                                 78    119     42    130   
...                                              ...    ...    ...    ...   
11008008       Berlin-Neukölln                     0      0     41     27   
11009009       Berlin-Treptow-Köpenick             0      0    110    181   
11010010       Berlin-Marzahn-Hellersdorf          0      0    139    124   
11011011       Berlin-Lichtenberg                  0      0     54     43   
11012012       Berlin-Reinickendorf                0      0     74     60   

                                                 fossils                  \
year                                        2020    2016    2017    2018   
ort_ags  ort_name                                                          
0        Deutschland                       97269  156629  153581  154380   
1          Schleswig-Holstein               3404   10319    8044    8327   
2          Hamburg                           613    6484    6123    8685   
3          Niedersachsen                    5775   21807   19646   18937   
4          Bremen                             86    1505    1266    1763   
...                                          ...     ...     ...     ...   
11008008       Berlin-Neukölln                15       0       0     700   
11009009       Berlin-Treptow-Köpenick       301       0       0    2065   
11010010       Berlin-Marzahn-Hellersdorf    172       0       0     741   
11011011       Berlin-Lichtenberg             48       0       0    1643   
11012012       Berlin-Reinickendorf           39       0       0     188   

                                                           ...  \
year                                         2019    2020  ...   
ort_ags  ort_name                                          ...   
0        Deutschland                       159026  163855  ...   
1          Schleswig-Holstein                9221    9306  ...   
2          Hamburg                           8392    9838  ...   
3          Niedersachsen                    20216   22023  ...   
4          Bremen                            1713    1340  ...   
...                                           ...     ...  ...   
11008008       Berlin-Neukölln                721     485  ...   
11009009       Berlin-Treptow-Köpenick       2483    1458  ...   
11010010       Berlin-Marzahn-Hellersdorf    1814    1643  ...   
11011011       Berlin-Lichtenberg            1758    1916  ...   
11012012       Berlin-Reinickendorf           245     402  ...   

                                          Fernwärme/Fernkälte                \
year                                                     2016   2017   2018   
ort_ags  ort_name                                                             
0        Deutschland                                    42350  43993  50748   
1          Schleswig-Holstein                            2427   1774   2191   
2          Hamburg                                       3529   2545   3865   
3          Niedersachsen                                 1427   1606   1453   
4          Bremen                                         603    394    544   
...                                                       ...    ...    ...   
11008008       Berlin-Neukölln                              0      0    279   
11009009       Berlin-Treptow-Köpenick                      0      0   1138   
11010010       Berlin-Marzahn-Hellersdorf                   0      0    403   
11011011       Berlin-Lic

In [10]:
#collapse levels of column names and join energy type to year
piv_df.columns = piv_df.columns.map('_'.join)

#prepend all columns with indicator name
piv_df=piv_df.add_prefix('buildings_newbuilding_energy_')

#reset index to recreate AGS and Name columns
piv_df.reset_index(inplace=True)

#rename Name and AGS columns and strip whitespace
piv_df.rename(columns={'ort_name':'Name','ort_ags':'AGS'},inplace=True)
piv_df['Name']=piv_df['Name'].str.strip()

#set AGS as index
piv_df.set_index('AGS',inplace=True)

piv_df

,Name,buildings_newbuilding_energy_renewables_2016,buildings_newbuilding_energy_renewables_2017,buildings_newbuilding_energy_renewables_2018,buildings_newbuilding_energy_renewables_2019,buildings_newbuilding_energy_renewables_2020,buildings_newbuilding_energy_fossils_2016,buildings_newbuilding_energy_fossils_2017,buildings_newbuilding_energy_fossils_2018,buildings_newbuilding_energy_fossils_2019,...,buildings_newbuilding_energy_Fernwärme/Fernkälte_2016,buildings_newbuilding_energy_Fernwärme/Fernkälte_2017,buildings_newbuilding_energy_Fernwärme/Fernkälte_2018,buildings_newbuilding_energy_Fernwärme/Fernkälte_2019,buildings_newbuilding_energy_Fernwärme/Fernkälte_2020,buildings_newbuilding_energy_Sonstige Heizenergie_2016,buildings_newbuilding_energy_Sonstige Heizenergie_2017,buildings_newbuilding_energy_Sonstige Heizenergie_2018,buildings_newbuilding_energy_Sonstige Heizenergie_2019,buildings_newbuilding_energy_Sonstige Heizenergie_2020
AGS,,,,,,,,,,,,,,,,,,,,,
0,Deutschland,64822,75208,83937,87137,97269,156629,153581,154380,159026,...,42350,43993,50748,55745,60312,911,821,1686,1352,1041
1,Schleswig-Holstein,2161,2644,2519,2876,3404,10319,8044,8327,9221,...,2427,1774,2191,2843,3053,113,25,185,140,148
2,Hamburg,562,550,874,656,613,6484,6123,8685,8392,...,3529,2545,3865,3722,5775,96,80,423,500,349
3,Niedersachsen,4011,4348,5150,5363,5775,21807,19646,18937,20216,...,1427,1606,1453,1958,2556,131,66,124,194,72
4,Bremen,78,119,42,130,86,1505,1266,1763,1713,...,603,394,544,693,579,2,2,16,28,12
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
11008008,Berlin-Neukölln,0,0,41,27,15,0,0,700,721,...,0,0,279,560,452,0,0,0,0,0
11009009,Berlin-Treptow-Köpenick,0,0,110,181,301,0,0,2065,2483,...,0,0,1138,1759,778,0,0,0,80,5
11010010,Berlin-Marzahn-Hellersdorf,0,0,139,124,172,0,0,741,1814,...,0,0,403,1268,1473,0,0,3,0,0


In [11]:
#save dataframe to file
piv_df.to_csv("data/final_data_landkreis_newbuildings_renewables.csv")